### Introduction

The aim is to build a machine learning model to automatically turn grayscale images into colored images. The model will be built from scratch (using PyTorch) i.e. without using any pre trained model.

At the end, the model will be able to colorize grayscale (or black and white) images. 

### Overview 

In image colorization, the goal is to produce a colored image given a grayscale input image. This problem is challenging because it is multimodal -- a single grayscale image may correspond to many plausible colored images. As a result, traditional models often relied on significant user input alongside a grayscale image. 

#### The Problem

The aim is to infer a full-colored image, which has 3 values per pixel (lightness, saturation, and hue), from a grayscale image, which has only 1 value per pixel (lightness only). 
To keep things simple for the time being, the model will only work with images of size $256 \times 256$, so our inputs are of size $256 \times 256 \times 1$ (the lightness channel) and our outputs are of size  $256 \times 256 \times 2$ (the other two channels). In future, the aim is to improve the framework so that it can work for images of any resolution.


Rather than work with images in the RGB format, as people usually do, we will work with them in the [LAB colorspace](https://en.wikipedia.org/wiki/CIELAB_color_space) ($L$ightness, $A$, and $B$) . This colorspace contains exactly the same information as RGB, but it will make it easier for us to separate out the lightness channel from the other two (which we call $A$ and $B$). We'll try to predict the color values of the input image directly (we call this regression).

#### The Data

Getting the data for colorisation problem is not as difficult as other problems as grayscale images can be found anywhere! :p.
For this project, we'll use a subset of the [MIT Places](http://places.csail.mit.edu/) dataset of places, landscapes, and buildings. 

In [ ]:
# Download the data(2.2GB)
!wget http://data.csail.mit.edu/places/places205/testSetPlaces205_resize.tar.gz

#unzip
!tar -xzf testSetPlaces205_resize.tar.gz

--2018-05-15 13:31:42--  http://data.csail.mit.edu/places/places205/testSetPlaces205_resize.tar.gz
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.35
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2341250899 (2.2G) [application/octet-stream]
Saving to: ‘testSetPlaces205_resize.tar.gz’

 testSetPlaces205_r  12%[=>                  ] 280.81M  5.53MB/s    eta 4m 53s 

              testS  99%[==================> ]   2.18G  5.07MB/s    eta 1s     

testSetPlaces205_re 100%[===================>]   2.18G  5.06MB/s    in 6m 33s  

2018-05-15 13:38:15 (5.67 MB/s) - ‘testSetPlaces205_resize.tar.gz’ saved [2341250899/2341250899]



In [ ]:
# Move data into training and validation directories
import os
os.makedirs('images/train/class/', exist_ok=True) # 40,000 images
os.makedirs('images/val/class/', exist_ok=True)   #  1,000 images
for i, file in enumerate(os.listdir('testSet_resize')):
  if i < 1000: # first 1000 will be val
    os.rename('testSet_resize/' + file, 'images/val/class/' + file)
  else: # others will be train
    os.rename('testSet_resize/' + file, 'images/train/class/' + file)

In [ ]:
# Make sure the images are there
from IPython.display import Image, display
display(Image(filename='images/val/class/84b3ccd8209a4db1835988d28adfed4c.jpg'))

#### Dependecies

We'll build and train our model with PyTorch. We'll also use torchvision, a helpful set of tools for working with images and videos in PyTorch, and scikit-learn for converting between RGB and LAB colorspces. 


In [ ]:
# Download and import libraries
!pip install pillow==4.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 5.3 MB/s 
     |████████████████████████████████| 112 kB 63.3 MB/s 
  Created wheel for pillow: filename=Pillow-4.3.0-cp37-cp37m-linux_x86_64.whl size=1064661 sha256=3145c55261d9c89c1882a5807fc513ddd631a13a135d9c20d4dfefef97f21f50
  Stored in directory: /root/.cache/pip/wheels/77/77/72/045aa57329073d3cdbd822995f8228c95d17cce12f1ca75f8f
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35432 sha256=db8cdd16378ce1688ef90748f337b1c8a32ec97a45ff0b141b84b097ad70e2ac
  Stored in directory: /root/.cache/pip/wheels/84/53/e6/37d90ccb3ad1a3ca98d2b17107e9fda401a7c541ea1eb6a65a
Successfully built pillow olefile
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into accou

In [ ]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision.models as models
from torchvision import datasets, transforms
# For utilities
import os, shutil, time
from PIL import Image, ImageOps

In [ ]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()


### The Model

Our model is a convolutional neural network. We first apply a number of convolutional layers to extract features from our image, and then we apply deconvolutional layers to upscale (increase the spacial resolution) of our features.  

Specifically, the beginning of our model will be ResNet-18, an image classification network with 18 layers and residual connections. We will modify the first layer of the network so that it accepts grayscale input rather than colored input. Only first 6 layers of the ResNet-18 will be used for our purpose.


Now, we'll define our model in code. We'll start with the second half of the net, the upsampling layers:

In [ ]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()
    MIDLEVEL_FEATURE_SIZE = 128

    ## First half: ResNet
    resnet = models.resnet18(num_classes=365) 
    # Change first conv layer to accept single-channel (grayscale) input
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Second half: Upsampling
    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

Now let's create our model and load it onto the GPU.

In [ ]:
model = ColorizationNet()

### Training

#### Loss Function

Since we are doing regression, we'll use a mean squared error loss function: we minimize the squared distance between the color value we try to predict, and the true (ground-truth) color value.  

In [ ]:
criterion = nn.MSELoss()

This loss function is slightly problematic for colorization due to the multi-modality of the problem. 

For example, if a gray dress could be red or blue, and our model picks the wrong color, it will be harshly penalized. As a result, our model will usually choose desaturated colors that are less likely to be "very wrong" than bright, vibrant colors. There has been [significant research](http://richzhang.github.io/colorization/) on this issue, but we will stick to our simple loss function for today.  



#### Optimizer

We will optimize our loss function (criterion) with the Adam optimizer. 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

#### Loading the data

We'll use torchtext to load the data. Since we want images in the LAB space, we first have to define a custom dataloader to convert the images.

In [ ]:
class GrayscaleImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is not None:
      img_original = self.transform(img)
      img_original = np.asarray(img_original)
      img_lab = rgb2lab(img_original)
      img_lab = (img_lab + 128) / 255
      img_ab = img_lab[:, :, 1:3]
      img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
      img_original = rgb2gray(img_original)
      img_original = torch.from_numpy(img_original).unsqueeze(0).float()
    if self.target_transform is not None:
      target = self.target_transform(target)
    return img_original, img_ab, target

Next we define transforms for our training and validation data.

In [ ]:
# Training
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip()])
train_imagefolder = GrayscaleImageFolder('images/train', train_transforms)
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=64, shuffle=True)

# Validation 
val_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])
val_imagefolder = GrayscaleImageFolder('images/val' , val_transforms)
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=64, shuffle=False)

#### Helper functions

Before we train, we define helper functions for tracking the training loss and converting images back to RGB. 

In [ ]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

#### Validation

In validation, we simply run model without backpropagation using `torch.no_grad`. 

In [ ]:
def validate(val_loader, model, criterion, save_images, epoch):
  model.eval()

  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  already_saved_images = False
  for i, (input_gray, input_ab, target) in enumerate(val_loader):
    data_time.update(time.time() - end)

    # Use GPU
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Run model and record loss
    output_ab = model(input_gray) # throw away class predictions
    loss = criterion(output_ab, input_ab)
    losses.update(loss.item(), input_gray.size(0))

    # Save images to file
    if save_images and not already_saved_images:
      already_saved_images = True
      for j in range(min(len(output_ab), 10)): # save at most 5 images
        save_path = {'grayscale': 'outputs/gray/', 'colorized': 'outputs/color/'}
        save_name = 'img-{}-epoch-{}.jpg'.format(i * val_loader.batch_size + j, epoch)
        to_rgb(input_gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    # Record time to do forward passes and save images
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to both value and validation
    if i % 25 == 0:
      print('Validate: [{0}/{1}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
             i, len(val_loader), batch_time=batch_time, loss=losses))

  print('Finished validation.')
  return losses.avg

#### Training

In training, run model and backpropagate using `loss.backward()`. We first define a function that trains for one epoch: 

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
  print('Starting training epoch {}'.format(epoch))
  model.train()
  
  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  for i, (input_gray, input_ab, target) in enumerate(train_loader):
    
    # Use GPU if available
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Record time to load data (above)
    data_time.update(time.time() - end)

    # Run forward pass
    output_ab = model(input_gray) 
    loss = criterion(output_ab, input_ab) 
    losses.update(loss.item(), input_gray.size(0))

    # Compute gradient and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record time to do forward and backward passes
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to value, not validation
    if i % 25 == 0:
      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
              epoch, i, len(train_loader), batch_time=batch_time,
             data_time=data_time, loss=losses)) 

  print('Finished training epoch {}'.format(epoch))

Next, we define a training loop and we train for 100 epochs:

In [ ]:
# Move model and loss function to GPU
if use_gpu: 
  criterion = criterion.cuda()
  model = model.cuda()

In [ ]:
# Make folders and set parameters
os.makedirs('outputs/color', exist_ok=True)
os.makedirs('outputs/gray', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 100

In [ ]:
# Train model
for epoch in range(epochs):
  # Train for one epoch, then validate
  train(train_loader, model, criterion, optimizer, epoch)
  with torch.no_grad():
    losses = validate(val_loader, model, criterion, save_images, epoch)
  # Save checkpoint and replace old best model if current model is better
  if losses < best_losses:
    best_losses = losses
    torch.save(model.state_dict(), 'checkpoints/model-epoch-{}-losses-{:.3f}.pth'.format(epoch+1,losses))

Starting training epoch 0


KeyboardInterrupt: ignored

### Results 

In [ ]:
# Show images 
import matplotlib.image as mpimg
image_pairs = [('outputs/color/img-2-epoch-0.jpg', 'outputs/gray/img-2-epoch-0.jpg'),
               ('outputs/color/img-7-epoch-0.jpg', 'outputs/gray/img-7-epoch-0.jpg')]
for c, g in image_pairs:
  color = mpimg.imread(c)
  gray  = mpimg.imread(g)
  f, axarr = plt.subplots(1, 2)
  f.set_size_inches(15, 15)
  axarr[0].imshow(gray, cmap='gray')
  axarr[1].imshow(color)
  axarr[0].axis('off'), axarr[1].axis('off')
  plt.show()

# Testing on a single image

In [ ]:
val_transforms1 = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])
iimage = Image.open('/content/wom.jpg')
iiorig = val_transforms1(iimage)
iiorig = np.asarray(iiorig)
img_lab = rgb2lab(iiorig)
img_lab = (img_lab + 128) / 255
img_ab = img_lab[:, :, 1:3]
img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
iiorig = rgb2gray(iiorig)
iiorig = torch.from_numpy(iiorig).unsqueeze(0).float()
iiorig = iiorig.reshape((1,1,224,224))

In [ ]:
output_ab = model(iiorig) 
save_path = {'grayscale': 'outputs/gray/', 'colorized': 'outputs/color/'}
save_name = 'wom.jpg'
to_rgb(iiorig[0].cpu(), ab_input=output_ab[0].detach().cpu(), save_path=save_path, save_name=save_name)

# Working with images of any dimensions (WORK IN PROGRESS!)

Idea:
1. Use sliding window (window size 256 x 256) on the input image. 
2. Make predictions on individual patches. 
3. Take the average of the overlapping area.
4. Recombine the patches to form a complete image.



In [ ]:
val_transforms1 = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])
iimage = Image.fromarray(patch_rnd)
iiorig = val_transforms1(iimage)
iiorig = np.asarray(iiorig)
# img_lab = rgb2lab(iiorig)
# img_lab = (img_lab + 128) / 255
# img_ab = img_lab[:, :, 1:3]
# img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
# iiorig = rgb2gray(iiorig)
iiorig = torch.from_numpy(iiorig).unsqueeze(0).float()
iiorig = iiorig.reshape((1,1,224,224))
output_ab = model(iiorig) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
def sliding_window(img, patch_size=224,istep=16, jstep=16):
  Ni = patch_size
  Nj = patch_size
  for i in range(0, img.shape[0] - Ni, istep):
    for j in range(0, img.shape[1] - Ni, jstep):
      patch = img[i:i + Ni, j:j + Nj]
      yield (i, j), patch

In [ ]:
def prep_img_pred(img_path):
  input_image = Image.open(img_path)
  input_gray = ImageOps.grayscale(input_image)
  test_image = np.asarray(input_gray)
  indices, patches = zip(*sliding_window(test_image))
  indices = np.asarray(indices)
  patches = np.asarray(patches)
  patches = patches.reshape((patches.shape[0], 1, patches.shape[1], patches.shape[2]))
  patches = torch.from_numpy(patches).float()
  return indices, patches

In [ ]:
test_indices, test_patches = prep_img_pred('/content/test.jpg')

In [ ]:
test_patches.shape

In [ ]:
iterations = test_patches.shape[0]//8

In [ ]:
iterations

In [ ]:
outputAB_from_model = model(test_patches[:8,:,:,:])
for i in range (1,iterations+1):
  st = i*8
  en = min(st+8,test_patches.shape[0])
  #print(st,en)
  if(st<test_patches.shape[0]):
    outputAB_from_model = torch.cat((outputAB_from_model,model(test_patches[st:en,:,:,:])),0)

In [ ]:
outputAB_from_model.shape

In [ ]:
input_image = Image.open('/content/test.jpg')
input_gray = ImageOps.grayscale(input_image)
test_image = np.asarray(input_gray)
print(test_image.shape)

In [ ]:
image_recons_skel = np.zeros((3,test_image.shape[0],test_image.shape[1]), dtype=np.float64)

In [ ]:
image_recons_skel.shape

In [ ]:
color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [ ]:
test_patches[342:343,:,:,:].shape

In [ ]:
color_image = torch.cat((test_patches[342:343,:,:,:][0], outputAB_from_model[0].detach()), 0).numpy()

In [ ]:
color_image.shape